In [90]:
from langchain.document_loaders import DirectoryLoader, CSVLoader, UnstructuredWordDocumentLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from sentence_transformers import SentenceTransformer, util
#from htmlTemplate import css, bot_template, user_template
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import create_history_aware_retriever
from langchain_huggingface import HuggingFaceEmbeddings

In [91]:
file_directory="data_directory" #the directory where documents are stored.
embedding_model='sentence-transformers/all-MiniLM-L6-v2' #the model identifier for Sentence Transformers, which will be used to generate text embeddings.
llm_model ="llama3.2" #specifies the version of the llm model used for generating responses in a chatbot setup.

In [92]:
def prepare_and_split_docs(directory):
    # Load the documents
    loaders = [
        DirectoryLoader(directory, glob="**/*.pdf",show_progress=True, loader_cls=PyPDFLoader),
        DirectoryLoader(directory, glob="**/*.txt",show_progress=True, loader_cls=TextLoader),
        DirectoryLoader(directory, glob="**/*.docx",show_progress=True),
        DirectoryLoader(directory, glob="**/*.csv",loader_cls=CSVLoader)
    ]


    documents=[]
    for loader in loaders:
        data =loader.load()
        documents.extend(data)

    # Initialize a text splitter
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,  
        chunk_overlap=256,
        disallowed_special=(),
        separators=["\n\n", "\n", " "]
    )

    # Split the documents and keep metadata
    split_docs = splitter.split_documents(documents)

    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs


In [93]:
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
def ingest_into_vectordb(split_docs):
    db = FAISS.from_documents(split_docs, embeddings)

    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    print("Documents are inserted into FAISS vectorstore")
    return db

In [94]:
def get_conversation_chain(retriever):
    llm = Ollama(model=llm_model)
    contextualize_q_system_prompt = (
        "Given the chat history and the latest user question, "
        "provide a response that directly addresses the user's query based on the provided documents. "
        "Do not rephrase the question or ask follow-up questions."
    )


    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


    ### Answer question ###
    system_prompt = (
        "As a personal chat assistant, provide accurate and relevant information based on the provided document in 2-3 sentences. "
      
        "{context}"
    )

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


    ### Statefully manage chat history ###
    store = {}


    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]


    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    print("Conversational chain created")
    return conversational_rag_chain

In [95]:
 # "Answer should be limited to 50 words and 2-3 sentences.  do not prompt to select answers or do not formualate a stand alone question. do not ask questions in the response. "
split_docs=prepare_and_split_docs(file_directory)
vector_db= ingest_into_vectordb(split_docs)
retriever =vector_db.as_retriever()
conversational_rag_chain=get_conversation_chain(retriever)


0it [00:00, ?it/s]
 98%|█████████▊| 50/51 [00:00<00:00, 8852.48it/s]
0it [00:00, ?it/s]


Documents are split into 50 passages
Documents are inserted into FAISS vectorstore
Conversational chain created


In [98]:
user_question="Who lives in Kansas?"

In [99]:

qa1=conversational_rag_chain.invoke(
    {"input": user_question},
    config={
        "configurable": {"session_id": "abc123"}
    }
)
print(qa1["answer"])

According to the provided document, Justin Xia lives in Kansas with Zach's grandmother. Unfortunately, the document does not provide information on Zach's family members or their relationship to Justin Xia beyond that.
